*Pyomo Optimization Problem Set, Spring 2022*
- Collin Guidry

In [2]:
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
from pyomo.environ import *
from pyomo.opt import SolverFactory

solver = SolverFactory("ipopt")    # loads the non-linear optimiser "IPOPT"

## Question

Hiland Appliance must determine how many TVs and Blu-Ray disc players to stock. It costs Hiland \\$1000 to purchase a TV and \\$200 to purchase a Blu-Ray player. A TV requires three square yards of storage space, and a Blu-ray disc player requires one square yard. The sale of a TV earns Hiland a profit of \\$150, and each Blu-ray disc player sale earns a profit of \\$50. Hiland has set the following goals (listed in order of importance):

- Goal 1: A maximum of \\$60,000 can be spent on purchasing TVs and Blu-ray disc players.
- Goal 2: Maximize profit from the sale of TVs and Blu-ray disc players.
- Goal 3: Minimize storage space for TVs and Blu-ray disc players.

### Part (a): Formulate the problem 

Formulate a multi-objective optimization problem that reflects the relative importance of the three goals, stating clearly your

- Decision variables
- Objective function
- Constraints

<span style="color:green"> 
**DV** 
- $x_1$= TV disc players to stock 
- $x_2$= Blue-Ray disc players to stock 
 

**OF**
- max profit = $f_1= 150x_1 + 50x_2$
- min storage = $f_2= 3x_1 +x_2$


**Constraints**
- $1000x_1 + 200x_2\leq 60000$ (budget)
- $x_1, x_2 \geq0$


### Part (b): Find Pareto-optimal points

Find at least one Pareto-optimal solution using Pyomo and IPOPT. Use comments to explain how your code reflects the relative importance of the three goals. 

In [5]:
# create a model
model = ConcreteModel()

# define variables
model.x1 = Var(bounds = (0, 100))
model.x2 = Var(bounds = (0, 300))

# define constraint
model.ConBudget = Constraint(expr = 1000*model.x1 + 200*model.x2 <= 60000)

# define constraint & objective
model.profit = Var()
model.space = Var()
model.ConProfit = Constraint(expr = model.profit == 150*model.x1 + 50*model.x2)
model.ConSpace = Constraint(expr = model.space == 3*model.x1 + model.x2)

# define objective
model.MaxProfit = Objective(expr = model.profit, sense = maximize)
model.MinSpace = Objective(expr = model.space, sense = minimize)               

In [6]:
# Lexicographic method
# deactivate MinSpace objective
model.MinSpace.deactivate()

# solve for maximized profit
solver.solve(model)

# save maximum profit value
profit_max = value(model.profit)

# add constraint for profit. Do not allow this profit level to decrease
model.constr_profit = Constraint(expr = model.profit >= profit_max)

# deactivate MaxProfit objective
model.MaxProfit.deactivate()

# activate MinSpace objective
model.MinSpace.activate()

# solve for minimized space
solver.solve(model)

# store values
profit = value(model.profit)
space = value(model.space)

# 
tvs = value(model.x1)
blu_ray = value(model.x2)

total_cost = 1000*tvs + 200*blu_ray

In [7]:
print("Total Cost: {:.2f}".format(total_cost) )
print("Maximum possible profit: {:.2f}".format(profit_max) )
print('\n')
print("Pareto optimal profit: {:.2f}".format(profit) )
print("Pareto optimal space: {:.2f}".format(space) )
print('\n')
print("TVs Purchased: {:.2f}".format(tvs) )
print("Blueray players purchased: {:.2f}".format(blu_ray) )

Total Cost: 60000.00
Maximum possible profit: 15000.00


Pareto optimal profit: 15000.00
Pareto optimal space: 300.00


TVs Purchased: 0.00
Blueray players purchased: 300.00
